In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.layers import Dense, Input, Conv2DTranspose, Concatenate,Conv2D,MaxPooling2D,Flatten,Reshape
from keras.models import Sequential, Model
from src.data_loader import Shifted_Data_Loader
from src.models import ConvGVAEBuilder

Using TensorFlow backend.


In [ ]:
class Shifted_Generator():
    def __init__(self,scale=2,translation=None,rotation=None):
        self.scale
        self.rotation=rotation
        self.translation=translation
        
        if self.rotation is not None:
            self.rotation=float(self.rotation)
            assert self.rotation >= 0.0
            assert self.rotation <= 1.0
        
        if self.translation is not None:
            self.translation=float(self.translation)
            assert self.translation >= 0.
            assert self.translation <= 1.
    
    def transform_im(self,im_stack,output,delta):
        num_im = len(im_stack)
        for i in trange(num_im):
            letter = im_stack[i]
            if self.rotation is not None:
                letter,rot = self.rotate_image(letter,rot_max=self.rotation)
            else:
                new_im = letter
                rot = [0]
            
            new_im,offsets = self.shift_image(letter,max_translation=self.translation)
                
            output[i] = np.reshape(new_im,(1,)+self.input_shape)
            offsets.extend(rot)
            delta[i] = np.array(offsets)
    
    def __call__(self):
        
        

In [2]:
DL = Shifted_Data_Loader(dataset='mnist',flatten=False,
                         rotation=None,
#                          translation=None,
                        )

input_shape:  (56, 56)
dataset:  mnist
scale:  2
tx_max:  0.75
rot_max:  None
loading mnist...


  5%|▍         | 2817/60000 [00:00<00:02, 28159.69it/s]

sx_train:  (60000, 56, 56)
making training data...


 35%|███▌      | 3533/10000 [00:00<00:00, 35323.97it/s]

making testing data...


100%|██████████| 10000/10000 [00:00<00:00, 28180.56it/s]


In [3]:
x_test = DL.x_test


In [4]:
builder = ConvGVAEBuilder()
mod = builder.build((28,28,1))
mod.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 784)          0           input[0][0]                      
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 500)          392500      flatten_1[0][0]                  
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 500)          250500      dense_1[0][0]                    
__________________________________________________________________________________________________
y_lat (Den

In [5]:
import keras.backend as K
K.gradients()

TypeError: gradients() missing 2 required positional arguments: 'loss' and 'variables'

In [ ]:
model = Sequential()
#convolutional layer with rectified linear unit activation
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28,28,1)))
#32 convolution filters used each of size 3x3
#again
model.add(Conv2D(64, (3, 3), activation='relu'))
#64 convolution filters used each of size 3x3
#choose the best features via pooling
model.add(MaxPooling2D(pool_size=(2, 2)))
#flatten since too many dimensions, we only want a classification output
model.add(Flatten())
#fully connected to get all relevant data
model.add(Dense(128, activation='relu'))

#output a softmax to squash the matrix into output probabilities
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [ ]:
model.summary()

In [ ]:
ch = 8
num_classes=10
z_dim = 3

inputs = Input(shape=(num_classes+z_dim,))